## Definicion del contexto

El contexto con el que quiero trabajar mi tarea es un proyecto personal en el que me encuentro trabajando el cual corresponde a un bot de trading algorítmico basado en señales. Éste toma estrategias de inversion usando indicadores técnicos y genera señales de compra y venta que ejecuta en los mercados a los que se encuentra conectado mediante API rest o recibiendo datos de manera ágil usando websockets de algunos comercios.

![TradingBot](TradingBot.png)

# Identificacion de requerimientos asociados al caso de uso:
Dentro de los posibles requerimientos asociados al caso de uso tenemos: (Click para abrir cada item)

<details> 
<summary>Problema 1</summary> 

- **Problema:** El bot debe integrarse con múltiples APIs de exchanges (Binance, Coinbase, Kraken), cada una con su propia estructura y formato de datos.
- **Patrón propuesto:** Adapter 
- **Justificación:** Permite adaptar interfaces incompatibles al modelo estándar del bot (`ExchangeClient`), facilitando la integración con múltiples exchanges sin modificar el núcleo de la aplicación. 
</details>

<details> 
<summary>Problema 2</summary> 

- **Problema:** Se desea añadir funcionalidades adicionales a las estrategias de trading (como logging, validación o métricas) sin modificar sus clases base.
- **Patrón propuesto:** Decorator 
- **Justificación:** Permite extender dinámicamente el comportamiento de estrategias envolviéndolas con clases adicionales, manteniendo el principio de abierto/cerrado y mejorando la reutilización.
</details>

<details>
<summary>Problema 3</summary> 

- **Problema:** El bot debe acceder a datos de mercado provenientes de diversas fuentes (API REST, WebSocket, base de datos), y tratarlos como una única fuente unificada. 
- **Patrón propuesto:** Facade 
- **Justificación:** Proporciona una interfaz simplificada para un conjunto complejo de subsistemas de adquisición de datos, ocultando la complejidad y mejorando la experiencia del desarrollador.
</details>

<details>
<summary>Problema 4</summary> 

- **Problema:** Algunas estrategias combinan varias señales (como RSI, MACD y volumen) que deben evaluarse de forma jerárquica o conjunta. 
- **Patrón propuesto:** Composite 
- **Justificación:** Permite tratar señales individuales y compuestas de forma uniforme, lo cual facilita la construcción y evaluación de estrategias complejas a partir de componentes simples. 
</details>

<details>
<summary>Problema 5</summary> 

- **Problema:** Los modelos de IA o configuraciones pesadas del bot se usan repetidamente y requieren optimización de memoria. 
- **Patrón propuesto:** Flyweight 
- **Justificación:** Permite reutilizar instancias compartidas de objetos pesados, reduciendo el consumo de memoria y mejorando el rendimiento general del sistema.
</details>

<details>
<summary>Problema 6</summary> 

- **Problema:** El bot debe enviar notificaciones a distintos canales (email, Telegram, consola, API), pero sin acoplar la lógica de generación de alertas con los medios de salida. 
- **Patrón propuesto:** Bridge 
- **Justificación:** Separa la abstracción de la implementación, permitiendo combinar libremente distintos tipos de notificaciones con múltiples canales sin modificar las clases existentes.
</details>

<details>
<summary>Problema 7</summary> 

- **Problema:** Se necesita controlar y restringir el acceso a ciertos recursos sensibles del bot (como la configuración en tiempo de ejecución o la ejecución de órdenes reales). 
- **Patrón propuesto:** Proxy 
- **Justificación:** Permite interponer una capa de control de acceso, logging o caching antes de delegar las acciones al objeto real, mejorando la seguridad y el control del sistema.
</details>

## Contexto integrado a la tarea
Para este caso usaremos el escenario propuesto en el problema 4
- **Evaluación jerarquica de señales de indicadores de trading:** En este caso se nos presenta una alternativa que nos permite programar de manera controlada el orden en el que se construyen las estrategias de trading que manejan múltiples señales, en particular estas se aproximan a un caso mas realista que lo que se presentó en la tarea 1 ya que normalmente al construir una señal de trading, como mínimo se manejan 3 categorías de indicadores/señales como lo son los osciladores (RSI, MACD, Stoch), las superopsiciónes (MA,EMA, BB) y las señales de acción del precio (cruces por soportes/resistencias, patrones de velas, volumen).

## Implementacion **Evaluación jerarquica de señales de indicadores de trading**

Para este caso compondremos una señal de trading de compra, que usa tres señales:
1. **Señal de tendencia:** Usaremos una regresion lineal de los precios de cierre para calcular la pendiente.
2. **Señal de momentum:** Usaremos el RSI para determinar si el precio se encuentra en un punto de reversion a la media (punto en el que existe mucha probabilidad de abandonar la tendencia de manera momentánea)
3. **Señal del precio:** Usaremos un indicador que compare si, al cierre de la vela actual, el cuerpo de la vela posee un cuerpo mas grande que el cuerpo de la vela anterior.

El orden de ejecucion de las señales será en el orden numérico establecido (primero probamos si se cumple 1, luego si se cumple 2 y finalmente si se cumple 3)

Una gran ventaja de esta implementación se dá en caso de que queramos darle mas detalle a cada señal, en ese caso no hay necesidad de modificar ninguna señal sino que podemos crear una extensión la señal en cuestion, donde se da mas detalle de lo que se quiere conseguir con la señal. Por ejemplo podríamos refinal la **Señal del precio** de manera que ya no solamente mire el cuerpo de las velas sino tambien el volumen del cierre (por ejemplo el volumen mayor al cierto % del volumen de la vela anterior)

![DiagramaComposite](DiagramaComposite.svg)

In [1]:
from abc import ABC, abstractmethod

# Interfaz o clase base para todas las señales de trading
class BuyTradingSignal(ABC):
    @abstractmethod
    def evaluate(self):
        """Evalúa la señal y retorna un valor booleano representando que cumple su condicion."""
        pass

# Clases Leaf que representan señales individuales

class SlopeTrendSignal(BuyTradingSignal):
    """Señal de tendencia"""
    def __init__(self, kline_window:int = 20, threshold:float = 0.05):# Se asume que se evaluan las ultimas 20 velas y un margen de aceptacion del 5%
        self.kline_window = kline_window
        self.threshold = threshold
        self.slope = None

    def evaluate(self):
        print("Evaluando tendencia")

        if self.slope>self.threshold:
            print("Tendencia alcista")
            return True
        
        return False
        
    def calculate_slope(self, klines):
        print("Calculando pendiente")
        self.slope = 0.06 #Para efectos del ejemplo asumiremos tendencia alcista con un indice de 0.6% por vela
        pass

class RSISignal(BuyTradingSignal):
    """Señal de momentum"""
    def __init__(self, kline_window: int = 20 ,rsi_upper: float = 80.0, rsi_lower: float = 20.0):# Se asume un margen sobre 80 y bajo 20, se evaluan las ultimas 20 velas
        self.kline_window = kline_window
        self.rsi_upper = rsi_upper
        self.rsi_lower = rsi_lower
        self.rsi_value = None

    def evaluate(self):
        print("Evaluando RSI")
        if self.rsi_value < self.rsi_lower:
            print("RSI en sobreventa")
            return True
        return False
    
    def calculate_rsi(self, klines):
        print("Calculando RSI")
        self.rsi_value = 16.0 # Para efectos del ejemplo asumiremos un RSI de 16.0

class PriceActionSignal(BuyTradingSignal):
    """Señal de accion del precio"""
    def __init__(self, threshold: float = 1.2):# Se asume que el cierre de la vela actual tiene un cuerpo 1.2 veces mayor que el cuerpo de la vela anterior
        self.threshold = threshold
        self.candle_body1 = None #Vela cierre actual
        self.candle_body2 = None #Vela cierre anterior

    def evaluate(self):
        print("Evaluando accion del precio")
        if self.candle_body1 > self.candle_body2 * self.threshold:
            print(r"Vela cierra con cuerpo superior al 20% del cuerpo anterior")
            return True
        return False
    
    def calculate_candle_body(self, klines):
        print("Calculando el cuerpo de las velas")
        # Suponiendo que el cuerpo de la vela es el cierre menos la apertura
        self.candle_body1 = 2
        self.candle_body2 = 1


In [10]:

# Componente Composite que puede contener múltiples señales
from typing import List


class CompositeSignal(BuyTradingSignal):
    def __init__(self):
        self._signals: List[BuyTradingSignal] = []

    def add_signal(self, signal: BuyTradingSignal):
        self._signals.append(signal)

    def remove_signal(self, signal: BuyTradingSignal):
        self._signals.remove(signal)

    def evaluate(self):
        print(f"\n Iniciando evaluacion de señales compuestas")
        print("--------------------------------------------")
        
        aggregated_value = True
        for signal in self._signals:
            print(f"Evaluando {signal.__class__.__name__}")
            aggregated_value &= signal.evaluate()
            print("--------------------------------------------")
        return aggregated_value

# Ejemplo de uso
if __name__ == "__main__":
    # Simulación de datos de velas (klines)
    klines = {
  "kline_1": {"timestamp": "2025-04-15T12:00:00", "open": 158.24, "high": 162.11, "low": 157.73, "close": 159.92, "volume": 758.34},
  "kline_2": {"timestamp": "2025-04-15T12:05:00", "open": 159.92, "high": 161.00, "low": 158.68, "close": 159.18, "volume": 482.67},
  "kline_3": {"timestamp": "2025-04-15T12:10:00", "open": 159.18, "high": 161.62, "low": 158.85, "close": 160.74, "volume": 659.21},
  "kline_4": {"timestamp": "2025-04-15T12:15:00", "open": 160.74, "high": 164.30, "low": 160.21, "close": 163.09, "volume": 534.56},
  "kline_5": {"timestamp": "2025-04-15T12:20:00", "open": 163.09, "high": 165.34, "low": 162.87, "close": 164.45, "volume": 912.42},
  "kline_6": {"timestamp": "2025-04-15T12:25:00", "open": 164.45, "high": 167.12, "low": 163.76, "close": 166.88, "volume": 435.78},
  "kline_7": {"timestamp": "2025-04-15T12:30:00", "open": 166.88, "high": 169.22, "low": 166.35, "close": 167.54, "volume": 678.90},
  "kline_8": {"timestamp": "2025-04-15T12:35:00", "open": 167.54, "high": 170.10, "low": 166.89, "close": 169.12, "volume": 520.33},
  "kline_9": {"timestamp": "2025-04-15T12:40:00", "open": 169.12, "high": 170.85, "low": 168.70, "close": 170.24, "volume": 610.75},
  "kline_10": {"timestamp": "2025-04-15T12:45:00", "open": 170.24, "high": 173.00, "low": 169.82, "close": 172.18, "volume": 790.15}
}

    # Crear instancias de señales individuales
    slope_signal = SlopeTrendSignal(20, 0.05)  # Ej. pendiente de tendencia
    slope_signal.calculate_slope(klines)  # Calcular la pendiente de tendencia

    rsi_signal = RSISignal(20, 80.0, 20.0)  # Ej. RSI con límites superior e inferior
    rsi_signal.calculate_rsi(klines)  # Calcular el RSI

    price_action_signal = PriceActionSignal(1.2)  # Ej. acción del precio con un umbral
    price_action_signal.calculate_candle_body(klines)  # Calcular el cuerpo de las velas


    # Crear una señal compuesta que combine múltiples señales
    combined_signal = CompositeSignal()
    combined_signal.add_signal(slope_signal)
    combined_signal.add_signal(rsi_signal)
    combined_signal.add_signal(price_action_signal)

    # Evaluar la señal compuesta
    result = combined_signal.evaluate()
    print(f"Señal de compra al cierre: {result}")

Calculando pendiente
Calculando RSI
Calculando el cuerpo de las velas

 Iniciando evaluacion de señales compuestas
--------------------------------------------
Evaluando SlopeTrendSignal
Evaluando tendencia
Tendencia alcista
--------------------------------------------
Evaluando RSISignal
Evaluando RSI
RSI en sobreventa
--------------------------------------------
Evaluando PriceActionSignal
Evaluando accion del precio
Vela cierra con precio superior al 20% del cierre anterior
--------------------------------------------
Señal de compra al cierre: True
